In [186]:
import itertools
import random

class Schelling:
    
    
    def __init__(self, width, height, empty_ratio, similarity_threshold, n_iterations, races):
        self.width=width
        self.height=height
        self.empty_ratio=empty_ratio
        self.similarity_threshold=similarity_threshold
        self.n_iterations=n_iterations
        self.races=races
        
    def population(self):
        self.empty_houses=[]
        self.agents={}
        self.a=list(range(1,self.width+1))
        self.b=list(range(1,self.height+1))
        self.all_houses=list(itertools.product(self.a,self.b))                     
        random.shuffle(self.all_houses)
        self.n_empty=round(self.empty_ratio*len(self.all_houses))
        self.remaining_houses=self.all_houses[:len(self.all_houses)-self.n_empty]
        self.race_list=list(range(1,self.races+1))
        for i in range(len(self.remaining_houses)):   
            random.shuffle(self.race_list)
            self.chosen_one=self.race_list[0]
            self.agents[self.remaining_houses[i]]=self.chosen_one
        return self.agents
        
                             
                            
        
        
      
        

In [203]:
trial=Schelling(3,3,0.2,0.8,44,100)
print(trial.population())

{(2, 2): 67, (2, 3): 3, (1, 1): 68, (1, 3): 2, (3, 2): 28, (1, 2): 36, (2, 1): 31}


In [150]:
round(6.6)


7

In [ ]:
  import itertools
        itertools.product(list1,list2)
        import random
        random.shuffle(list)

In [114]:
import random

d = [1, 2, 3, 4, 5, 6, 7]
random.shuffle(d)

print(d)

[3, 6, 1, 2, 7, 5, 4]
